<a href="https://colab.research.google.com/github/kimbaorrr/colab_exercises/blob/main/Other%20Notebooks/Yolov8_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thiết lập chung

In [ ]:
# Thiết đặt đường dẫn thư mục gốc
import os

HOME = os.getcwd()
print(HOME)

In [ ]:
# Cài đặt & nạp các thư viện cần thiết
!pip install -U ultralytics roboflow

import ultralytics
print(ultralytics.checks())

from ultralytics import YOLO
from IPython.display import display, Image
import glob
import random
from google.colab.patches import cv2_imshow
import math
import os
import numpy as np

import torch
print(f'=> You have {torch.cuda.device_count()} GPU !')

# Train từ bộ dữ liệu Roboflow


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# Tải bộ dữ liệu từ Roboflow

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="nbernU2pi3cCX5AH4VTi")
project = rf.workspace("my-workspace-jlq0f").project("motorbike-license-plate-detection")
dataset = project.version(6).download("yolov8")


In [ ]:
%cd {HOME}

# Bắt đầu Train
!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 plots=True batch=-1 seed=30 device=0 optimizer=AdamW

# Đánh giá quá trình Train
!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')
%load_ext tensorboard
%tensorboard --logdir {HOME}/runs

In [ ]:
%cd {HOME}

# Đánh giá mô hình qua tập Valid
!yolo task=detect mode=val model={HOME}'/runs/detect/train/weights/best.pt' data={dataset.location}/data.yaml save_json=True

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:5]:
      display(Image(filename=image_path, width=400))
      print('\n')

# Train từ bộ dữ liệu ngoài

In [ ]:
%cd '/content'

# Restore project to GDrive
!cp -r '/content/drive/MyDrive/yolo/v8/runs' ./
!cp -r '/content/drive/MyDrive/yolo/v8/datasets' ./

In [ ]:
%cd {HOME}/drive/MyDrive/yolo/v8/custom_training/my_dataset

## Training từ bộ dữ liệu ngoài
# Dọn dẹp tệp cũ
!rm -f 'train.txt' 'val.txt'

# Tách dữ liệu để Train/Test
val_split = 0.2
files = []
for ext in ["*.png", "*.jpeg", "*.jpg"]:
  image_files = glob.glob(os.path.join("data", ext))
  files += image_files

nb_val = math.floor(len(files)*val_split)
rand_idx = np.random.randint(0, len(files), nb_val)

# Tạo file train.txt
with open("train.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

# Tạo file val.txt
with open("val.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')



In [ ]:
%cd {HOME}

# Bắt đầu Train
!yolo task=detect mode=train model=yolov8l.pt data='/content/drive/MyDrive/yolo/v8/custom_training/my_dataset/data.yaml' optimizer='AdamW' epochs=50 imgsz=640 plots=True batch=-1 seed=20 device=0

# Đánh giá quá trình Train
!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')
%load_ext tensorboard
%tensorboard --logdir {HOME}/runs

# Python ENV + OpenCV

In [ ]:
# Python Environment
## Dự đoán & xuất tọa độ bboxs

model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')
results = model.predict(source='https://www.topgear.com/sites/default/files/2022/07/6_0.jpg', conf=0.25, save=True)
print(results[0].boxes.xyxy)
res_plotted = results[0].plot()
cv2_imshow(res_plotted) # Thay thế cv2.imshow('YOLO', res_plotted) khi chạy trên PC

In [ ]:
import cv2 as cv
from ultralytics import YOLO

# Nạp mô hình
model = YOLO(f'best.pt')
# Bắt các frame từ Video
cap = cv.VideoCapture('ny2.mp4')
cap.set(cv.CAP_PROP_BUFFERSIZE, 1)

while cap.isOpened():
    status, frame = cap.read()
    # Kiểm tra trạng thái cap
    if not status:
      break
	  # Nhấn ESC để thoát
    if cv.waitKey(33) == 27:
      break
    results = model.predict(frame)
    annotated_frame = results[0].plot()
    cv.imshow('YOLO Detection', annotated_frame)
cap.release()
cv.destroyAllWindows()

In [ ]:
import cv2 as cv
from ultralytics import YOLO

cap = cv.VideoCapture('file.mp4', cv.CAP_MSMF)
cap.set(cv.CAP_PROP_BUFFERSIZE, 1)
cap.set(cv.CAP_PROP_HW_ACCELERATION, cv.VIDEO_ACCELERATION_NONE)

model = YOLO('yolov8n.pt')
class_names = model.names

while cap.isOpened():
	cap.grab()
	status, frame = cap.read()
	# Kiểm tra trạng thái cam
	if not status:
		break
	# Nhấn ESC để thoát
	if cv.waitKey(33) == 27:
		break
	frame = cv.resize(frame, (1920, 1080))
	results = model.predict(frame, imgsz=640, stream=True, save_crop=True, augment=True)
	for result in results:
		for box in result.boxes.numpy():
			r = box.xyxy[0].astype(int)
			label = class_names[int(box.cls)]
			accuracy = '%.4f' % box.conf[0]
			# Đặt Text
			cv.putText(
				frame,
				f'{label} {accuracy}',
				(r[0], r[1] - 6),
				fontFace=cv.FONT_HERSHEY_COMPLEX,
				fontScale=0.6,
				color=(0, 255, 0),
				thickness=1
			)
			# Vẽ viền quanh đối tượng
			cv.rectangle(
				frame,
				r[:2],
				r[2:],
				color=(0, 0, 255),
				thickness=2
			)
	cv.imshow('YOLO Detection', frame)
cap.release()
cv.destroyAllWindows()


# Backup & Restore


In [ ]:
%cd '/content/drive/MyDrive/yolo/v8/license_plate'

# Backup project to GDrive
!cp -r '/content/runs' ./
!cp -r '/content/datasets' ./